## About Data
- **Name** : Flight Status Prediction
- **Source** : Kaggle (https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022)

##### Description
- This dataset contains all flight information including cancellation and delays by airline for dates back to January 2018 to year 2022. For your convenience you can use the Combined_Flights_XXXX.csv or Combined_Flights_XXXX.parquet files to access the combined data for the entire year.

##### Columns
- The DataSet has total of 61 columns and over 25M records.




### Importing Required Libraries

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, IntegerType
from pyspark.sql.functions import *

### Creating SparkSession with appName "Data603_FinalProject"

In [ ]:
spark = SparkSession.builder.appName('Data603_FinalProject').getOrCreate()

### Importing Data from DBFS (Here we are choosing .parquet files because size issues)

In [ ]:
df = spark.read.parquet('/FileStore/tables/*.parquet')

### Total Number of Records

In [ ]:
df.count()

Out[5]: 29193782

### DataFrame Overview

In [ ]:
display(df)

### Schema of the DataFrame

In [ ]:
df.printSchema()

root
 |-- FlightDate: timestamp (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: long (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: long (nullable = true)
 |-- Quarter: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- DayofMonth: long (nullable = true)
 |-- DayOfWeek: long (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |-- DOT_ID_Mar

# Data Cleaning

#### Finding Null Values

In [ ]:
def findMissing():
    from pyspark.sql.functions import col, isnan, when
    total_count = 0
    for column in df.columns:
        null_count = df[df[column].isNull()].count()
        total_count = total_count + null_count
        print(f"Number of null or NA values in column {column}: {null_count}")
    
    print(f'Total Null Values: {total_count}, Null Values Percentage: {total_count * 100/ df.count()}')

#### Count of Null Values by Column

In [ ]:
findMissing()

Number of null or NA values in column FlightDate: 0
Number of null or NA values in column Airline: 0
Number of null or NA values in column Origin: 0
Number of null or NA values in column Dest: 0
Number of null or NA values in column Cancelled: 0
Number of null or NA values in column Diverted: 0
Number of null or NA values in column CRSDepTime: 0
Number of null or NA values in column DepTime: 761652
Number of null or NA values in column DepDelayMinutes: 763084
Number of null or NA values in column DepDelay: 763084
Number of null or NA values in column ArrTime: 786177
Number of null or NA values in column ArrDelayMinutes: 846183
Number of null or NA values in column AirTime: 852561
Number of null or NA values in column CRSElapsedTime: 22
Number of null or NA values in column ActualElapsedTime: 845637
Number of null or NA values in column Distance: 0
Number of null or NA values in column Year: 0
Number of null or NA values in column Quarter: 0
Number of null or NA values in column Month: 

###### Filling null values of DepDelayMinutes as DepDelay and DepDelayMinutes will be 0 if there is no Delay. 
###### Note: We considered DepTime as NotNull because if incase the flight has canceled then it may not have DepTime at all.

In [ ]:
df = df.withColumn("DepDelayMinutes", when((col("DepDelay").isNull()) & (col("DepDelayMinutes").isNull()) & (col("DepTime").isNotNull()), 0).otherwise(col("DepDelayMinutes")))

df = df.withColumn("DepDelay", when((col("DepDelay").isNull()) & (col("DepDelayMinutes").isNull()) & (col("DepTime").isNotNull()), 0).otherwise(col("DepDelay")))

#### Null value count after replacing with 0

In [ ]:
null_count_depdelay_minutes = df.where(col("DepDelayMinutes").isNull()).count()
print(f"Count of null values in 'DepDelayMinutes': {null_count_depdelay_minutes}")

null_count_depdelay = df.where(col("DepDelay").isNull()).count()
print(f"Count of null values in 'DepDelay': {null_count_depdelay}")


Count of null values in 'DepDelayMinutes': 761632
Count of null values in 'DepDelay': 763084


#### Filling null values if DepTime is Null which means the flight has canceled

In [ ]:
df = df.withColumn("DepDelayMinutes", when((col("DepDelay").isNull()) & (col("DepDelayMinutes").isNull()) & (col("DepTime").isNull()), 0).otherwise(col("DepDelayMinutes")))

df = df.withColumn("DepDelay", when((col("DepDelay").isNull()) & (col("DepDelayMinutes") == 0) & (col("DepTime").isNull()), 0).otherwise(col("DepDelay")))

df = df.withColumn("DepTime", when((col("DepDelay") == 0) & (col("DepDelayMinutes") == 0) & (col("DepTime").isNull()), 0).otherwise(col("DepTime")))


null_count_depdelay_minutes = df.where(col("DepDelayMinutes").isNull()).count()
print(f"Count of null values in 'DepDelayMinutes': {null_count_depdelay_minutes}")

null_count_depdelay = df.where(col("DepDelay").isNull()).count()
print(f"Count of null values in 'DepDelay': {null_count_depdelay}")

null_count_deptime = df.where(col("DepTime").isNull()).count()
print(f"Count of null values in 'DepTime': {null_count_deptime}")

Count of null values in 'DepDelayMinutes': 0
Count of null values in 'DepDelay': 1452
Count of null values in 'DepTime': 0


#### Filling null values of DepDelay where CRSDepTime == DepTime. This means the flight is on time

In [ ]:
df = df.withColumn("DepDelay", when(col("CRSDepTime") == col("DepTime"), 0).otherwise(col("DepDelay")))

# Featuring Engineering

#### Creating Delayed and EarlyDep using DepDelay

In [ ]:
df = df.withColumn("Delayed", when((col("DepDelay") > 0) | (col("DepDelay") == 0), 0).otherwise(1))
df = df.withColumn("EarlyDep", when((col("DepDelay") < 0) | (col("DepDelay") == 0), 0).otherwise(1))

In [ ]:
display(df)

#### Encoding Cancelled and Diverted using StringIndexer

In [ ]:
df = df.withColumn("Cancelled", col("Cancelled").cast("string"))
df = df.withColumn("Diverted", col("Diverted").cast("string"))

In [ ]:
from pyspark.ml.feature import StringIndexer

# Create a StringIndexer
indexer_C = StringIndexer(inputCol="Cancelled", outputCol="Cancelled_Flight")

# Fit and transform the DataFrame
df = indexer_C.fit(df).transform(df)

In [ ]:
from pyspark.ml.feature import StringIndexer

# Create a StringIndexer
indexer_D = StringIndexer(inputCol="Diverted", outputCol="Diverted_Flight")

# Fit and transform the DataFrame
df = indexer_D.fit(df).transform(df)

In [ ]:
display(df)

#### Dropping unwanted Columns

In [ ]:
df = df.drop("Cancelled")
df = df.drop("Diverted")
df = df.drop("FlightDate")

In [ ]:
df = df.drop("DepDelayMinutes")
df = df.drop("DepDelay")

In [ ]:
df = df.drop("Operated_or_Branded_Code_Share_Partners")
df = df.drop("DOT_ID_Marketing_Airline")
df = df.drop("IATA_Code_Marketing_Airline")
df = df.drop("Flight_Number_Marketing_Airline")
df = df.drop("Operating_Airline")
df = df.drop("DOT_ID_Operating_Airline")
df = df.drop("IATA_Code_Operating_Airline")
df = df.drop("Tail_Number")
df = df.drop("Flight_Number_Operating_Airline")
df = df.drop("OriginAirportID")
df = df.drop("OriginAirportSeqID")
df = df.drop("OriginCityMarketID")
df = df.drop("OriginStateFips")
df = df.drop("OriginWac")
df = df.drop("DestAirportID")
df = df.drop("DestAirportSeqID")
df = df.drop("DestCityMarketID")
df = df.drop("DestStateFips")
df = df.drop("DestWac")
df = df.drop("DepTimeBlk")
df = df.drop("ArrTimeBlk")

In [ ]:
findMissing()

Number of null or NA values in column Airline: 0
Number of null or NA values in column Origin: 0
Number of null or NA values in column Dest: 0
Number of null or NA values in column CRSDepTime: 0
Number of null or NA values in column DepTime: 0
Number of null or NA values in column ArrTime: 786177
Number of null or NA values in column ArrDelayMinutes: 846183
Number of null or NA values in column AirTime: 852561
Number of null or NA values in column CRSElapsedTime: 22
Number of null or NA values in column ActualElapsedTime: 845637
Number of null or NA values in column Distance: 0
Number of null or NA values in column Year: 0
Number of null or NA values in column Quarter: 0
Number of null or NA values in column Month: 0
Number of null or NA values in column DayofMonth: 0
Number of null or NA values in column DayOfWeek: 0
Number of null or NA values in column Marketing_Airline_Network: 0
Number of null or NA values in column OriginCityName: 0
Number of null or NA values in column OriginSta

#### Creating 'AirTraffic' column based on total of flights that are scheduled on the same hour, day, month, year.

In [ ]:
df = df.withColumn("DepTime", col("DepTime").cast('string'))

df = df.withColumn("hour", when(length(col("DepTime")) == 6, col("DepTime").substr(1, 2)).otherwise(col("DepTime").substr(1, 1)))

In [ ]:
df = df.drop("DepTime")

In [ ]:
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("Origin", "hour", "DayofMonth", "Month", "Year")

df = df.withColumn("AirTraffic", count("*").over(windowSpec))

In [ ]:
df = df.drop("DivAirportLandings")

#### Displaying Final DataFrame

In [ ]:
display(df)